In [1]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
from typing import List

def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def articles_crawler(url:str) -> List[str]:
    #html 불러오기
    original_html = requests.get(url,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 모든 링크 수집
    all_links = [a.get("href") for a in html.find_all("a", href=True)]
    # 뉴스 링크 필터링
    news_links = [link for link in all_links if "n.news.naver.com" in link]
    news_links = list(set(news_links))  # 중복 제거

    return news_links



In [2]:
search = input("검색할 키워드를 입력해주세요:")
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")

page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  3 페이지


In [3]:
# URL 생성
url_list = makeUrl(search, page, page2)

# 결과 저장 리스트 초기화
news_titles = []
news_url_nested = []  # 중첩 리스트
news_contents = []

# 뉴스 URL 수집
for single_url in url_list:
    result = articles_crawler(single_url)
    news_url_nested.append(result)

# 중첩 리스트 평탄화 함수 정의
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 평탄화 실행
news_url_flat = []
makeList(news_url_flat, news_url_nested)





생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=물류&start=21']


['https://n.news.naver.com/mnews/article/421/0008285556?sid=154',
 'https://n.news.naver.com/mnews/article/001/0015418575?sid=101',
 'https://n.news.naver.com/mnews/article/277/0005600521?sid=103',
 'https://n.news.naver.com/mnews/article/421/0008285558?sid=154',
 'https://n.news.naver.com/mnews/article/023/0003908211?sid=101',
 'https://n.news.naver.com/mnews/article/082/0001328284?sid=102',
 'https://n.news.naver.com/mnews/article/014/0005356832?sid=101',
 'https://n.news.naver.com/mnews/article/003/0013274459?sid=101',
 'https://n.news.naver.com/mnews/article/018/0006026570?sid=103',
 'https://n.news.naver.com/mnews/article/421/0008285544?sid=154',
 'https://n.news.naver.com/mnews/article/030/0003317275?sid=103',
 'https://n.news.naver.com/mnews/article/421/0008282941?sid=103',
 'https://n.news.naver.com/mnews/article/421/0008285537?sid=154',
 'https://n.news.naver.com/mnews/article/016/0002478626?sid=101',
 'https://n.news.naver.com/mnews/article/015/0005138375?sid=103',
 'https://

In [4]:
# 네이버 뉴스 URL만 필터링
final_urls = []
for i in tqdm(range(len(news_url_flat))):
    if "n.news.naver.com" in news_url_flat[i]:
        final_urls.append(news_url_flat[i])
    else:
        pass


100%|██████████| 60/60 [00:00<00:00, 461758.24it/s]


In [ ]:

# 검색할 키워드를 입력해주세요:물류
# 크롤링할 시작 페이지를 입력해주세요.
# 크롤링할 시작 페이지: 1 페이지
# 크롤링할 종료 페이지를 입력해주세요.
# 크롤링할 종료 페이지: 3 페이지

In [5]:
for i in tqdm(final_urls):
    
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    for tag in news_html.select("em.img_desc"):
        tag.decompose()

    # 제목 크롤링
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    title = title.get_text(strip=True) if title else "제목 없음"

    # 본문 크롤링
    content = news_html.select_one("article#dic_area") or news_html.select_one("#articeBody")
    if content:
        content = content.get_text(separator="\n", strip=True)
    else:
        content = "본문 없음"

    # 태그 제거 및 전처리
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)

    content = re.sub(r"\n{3,}", "\n\n", content).strip()
    content = re.sub(r"<!--.*?-->", "", content, flags=re.DOTALL)


    # 리스트에 저장
    news_titles.append(title)
    news_contents.append(content)


100%|██████████| 60/60 [00:12<00:00,  4.70it/s]


In [6]:
print("검색된 기사 갯수: 총 ",len(final_urls),'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))

검색된 기사 갯수: 총  60 개

[뉴스 제목]
["'충주 항공물류 허브공항 유치를 위해'", '인도 물류기업 올카고, 부산항 신항 진출…물류센터 개장', '현대모비스, 경주에 통합물류센터 개소…하루 1만4000건 처리', "충주 항공물류 허브공항 유치 위해'", '현대모비스, 970억원 투입… AS 부품 전용 물류센터 열어', '20년 끈 김해 풍유물류단지, 또 표류하나', '부산항 신항에 인도·한국 합작 물류센터 개장', '하루 1만4000건 처리…현대모비스, 영남물류센터 가동', '현대모비스, 경주에 축구장 11개 크기 대규모 물류센터 개소', "'충주 항공물류 허브공항 유치를 위해'", '현대모비스, 경주 통합물류센터 본격 가동', '현대모비스, 두번째 물류센터 구축…현대차·기아 AS 더 빨라진다', "'항공물류 허브공항 유치를 위해'", '한국네트웍스, ‘디지털 제조 혁신’ 세미나 개최…스마트 물류 경쟁력 알려', '현대모비스, 970억 투입 영남물류센터 개소…국내 두 번째', '부산진해경자청, 로봇·수소 등 5대 전략품목 물류산업 활성화', '축구장 11개 크기… 현대모비스 두 번째 통합물류센터 활짝', '현대모비스, 전국에 현대차기아 ‘부품 공급’ 2번째 물류센터 개소', '현대모비스, 경주 물류센터 개소…현대차·기아 부품 공급', '인도계 물류기업 ‘올카고 그룹’ 부산 신항 진출', '현대모비스, 경주에 영남권 최대 A/S 부품 통합물류센터 본격 운영', "'항공물류 허브공항 유치를 위해'", '현대모비스, 경주에 축국장 11개 크기 통합물류센터 구축', '인도계 세계적 물류기업 올카고 그룹, 부산한 신항 진출', '현대모비스, 경주에 새 물류센터 지었다...내수용 AS부품 거점', '현대모비스, 경주에 국내 2번째 통합물류센터…축구장 11개 크기', '인도계 글로벌 물류기업 올카고 그룹, 부산신항 진출', '코람코 "서울 오피스 시장 상승세…물류센터도 회복 가시화"', "'충주 항공물류 허브공항 유치' 적힌 비행기 모형 받는 김문수 후보", 